In [ ]:
-- Informatics 1 Functional Programming
-- December 2015
-- SITTING 1 (09:30 - 11:30)

import Test.QuickCheck( quickCheck, 
                        Arbitrary( arbitrary ),
                        oneof, elements, sized, (==>), Property )
import Control.Monad -- defines liftM, liftM3, used below
import Data.List
import Data.Char

In [ ]:
p :: [Int] -> Int
p l = sum [x | x <- l, x > 0] `div` 60 `mod` 12 + 1

prop_p = 
    p [] == 1 &&
    p [-30, -20] == 1 &&
    p [20, -30, 30, 14, -20] == 2 &&
    p [200, 45] == 5 &&
    p [60, -100] == 2 &&
    p [60,-100,360,-20,240,59] == 12 &&
    p [60,-100,360,-20,240,60] == 1
    
quickCheck prop_p

In [ ]:
q :: [Int] -> Int
q l = mins l `div` 60 `mod` 12 + 1
    where mins [] = 0
          mins (x:xs) | x > 0     = (x + mins xs)
                      | otherwise = mins xs
    

prop_q = 
    q [] == 1 &&
    q [-30, -20] == 1 &&
    q [20, -30, 30, 14, -20] == 2 &&
    q [200, 45] == 5 &&
    q [60, -100] == 2 &&
    q [60,-100,360,-20,240,59] == 12 &&
    q [60,-100,360,-20,240,60] == 1
    
quickCheck prop_q

In [ ]:
r :: [Int] -> Int
r = ((+) 1) . (`mod` 12) . (`div` 60) . sum . filter (>0)

prop_r = 
    r [] == 1 &&
    r [-30, -20] == 1 &&
    r [20, -30, 30, 14, -20] == 2 &&
    r [200, 45] == 5 &&
    r [60, -100] == 2 &&
    r [60,-100,360,-20,240,59] == 12 &&
    r [60,-100,360,-20,240,60] == 1
    
quickCheck prop_r

In [ ]:
f :: String -> String    
f s = [x | (x, i) <- zip s [0..], i == 0 || (s !! (i-1) /= x)]

prop_f = 
    f "Tennessee" == "Tenese" &&
    f "llama" == "lama" &&
    f "oooh" == "oh" &&
    f "none here" == "none here" &&
    f "nNnor hEere" == "nNnor hEere" &&
    f "A" == "A" &&
    f "" == ""
    
quickCheck prop_f

-- A better solution for this question
f' :: String -> String
f' "" = ""
f' (x:xs) = x:[b | (a, b) <- zip (x:xs) xs, a /= b]

In [ ]:
g :: String -> String
g [] = []
g [x] = [x]
g (x:xs) | x == (head xs) = g xs
         | otherwise      = x:(g xs)
         
prop_g = 
    g "Tennessee" == "Tenese" &&
    g "llama" == "lama" &&
    g "oooh" == "oh" &&
    g "none here" == "none here" &&
    g "nNnor hEere" == "nNnor hEere" &&
    g "A" == "A" &&
    g "" == ""
    
quickCheck prop_g

In [ ]:
data Regexp = Epsilon
            | Lit Char
            | Seq Regexp Regexp
            | Or Regexp Regexp
        deriving (Eq, Ord)

-- turns a Regexp into a string approximating normal regular expression notation

showRegexp :: Regexp -> String
showRegexp Epsilon = "e"
showRegexp (Lit c) = [toUpper c]
showRegexp (Seq r1 r2) = "(" ++ showRegexp r1 ++ showRegexp r2 ++ ")"
showRegexp (Or r1 r2) = "(" ++ showRegexp r1 ++ "|" ++ showRegexp r2 ++ ")"

-- for checking equality of languages

equal :: Ord a => [a] -> [a] -> Bool
equal xs ys = sort xs == sort ys

-- For QuickCheck

instance Show Regexp where
    show  =  showRegexp

instance Arbitrary Regexp where
  arbitrary = sized expr
    where
      expr n | n <= 0 = oneof [elements [Epsilon]]
             | otherwise = oneof [ liftM Lit arbitrary
                                 , liftM2 Seq subform subform
                                 , liftM2 Or subform subform
                                 ]
             where
               subform = expr (n `div` 2)



r1 = Seq (Lit 'A') (Or (Lit 'A') (Lit 'A'))   -- A(A|A)
r2 = Seq (Or (Lit 'A') Epsilon)
         (Or (Lit 'A') (Lit 'B'))             -- (A|e)(A|B)
r3 = Seq (Or (Lit 'A') (Seq Epsilon
                            (Lit 'A')))
         (Or (Lit 'A') (Lit 'B'))             -- (A|(eA))(A|B)
r4 = Seq (Or (Lit 'A')
             (Seq Epsilon (Lit 'A')))
         (Seq (Or (Lit 'A') (Lit 'B'))
              Epsilon)                        -- (A|(eA))((A|B)e)
r5 = Seq (Seq (Or (Lit 'A')
                  (Seq Epsilon (Lit 'A')))
              (Or Epsilon (Lit 'B')))
         (Seq (Or (Lit 'A') (Lit 'B'))
              Epsilon)                        -- ((A|(eA))(e|B))((A|B)e)
r6 = Seq (Seq Epsilon Epsilon)
         (Or Epsilon Epsilon)                 -- (ee)(e|e)

In [ ]:
language :: Regexp -> [String]
language (Epsilon)   = [""]
language (Lit c1)    = [[c1]]
language (Seq r1 r2) = nub [l++r | l <- (language r1), r <- (language r2)]   
language (Or r1 r2)  = nub $ (language r1) ++ (language r2)

prop_language =
    equal (language r1) ["AA"] &&
    equal (language r2) ["AA", "AB", "A", "B"] &&
    equal (language r3) ["AA", "AB"] &&
    equal (language r4) ["AA", "AB"] &&
    equal (language r5) ["AA", "AB", "ABA", "ABB"] &&
    equal (language r6) [""]
        
quickCheck prop_language

In [ ]:
simplify :: Regexp -> Regexp
simplify (Seq r1 r2) | sr1 == Epsilon      = sr2
                     | sr2 == Epsilon      = sr1
                     | otherwise           = Seq sr1 sr2
                     where sr1 = simplify r1
                           sr2 = simplify r2
simplify (Or r1 r2)  | sr1 == sr2  = sr1
                     | otherwise   = Or sr1 sr2
                     where sr1 = simplify r1
                           sr2 = simplify r2

simplify r = r

prop_simplify_1 =
    simplify r1 == Seq (Lit 'A') (Lit 'A') &&
    simplify r2 == Seq (Or (Lit 'A') Epsilon) (Or (Lit 'A') (Lit 'B')) &&
    simplify r3 == Seq (Lit 'A') (Or (Lit 'A') (Lit 'B')) &&
    simplify r4 == Seq (Lit 'A') (Or (Lit 'A') (Lit 'B')) &&
    simplify r5 ==  Seq (Seq (Lit 'A')
                        (Or Epsilon (Lit 'B')))
                        (Or (Lit 'A') (Lit 'B')) &&
    simplify r6 == Epsilon
    
quickCheck prop_simplify_1

prop_simplify_2 :: Regexp -> Bool
prop_simplify_2 r = equal (language r) (language (simplify r))

quickCheck prop_simplify_2